In [10]:
import json
from pathlib import Path
from datetime import datetime, timedelta, timezone
from collections import defaultdict

def load_last_n_days(prefix, days, folder="."):
    now = datetime.now(timezone.utc)
    start_date = now - timedelta(days=days)
    merged = defaultdict(lambda: {"wins": 0, "losses": 0})

    folder_path = Path(folder)

    for file in folder_path.glob(f"{prefix}_*.json"):
        try:
            date_str = file.stem.split("_")[-1]
            file_date = datetime.strptime(date_str, "%Y-%m-%d").replace(tzinfo=timezone.utc)
            if file_date < start_date:
                continue

            with file.open("r") as f:
                data = json.load(f)

            for key, value in data.items():
                merged[key]["wins"] += value.get("wins", 0)
                merged[key]["losses"] += value.get("losses", 0)
        except Exception as e:
            print(f"Skipping file {file.name}: {e}")
    return merged

# Load data from last 5 days
winrate_data = load_last_n_days("winrate", 30)
synergy_data = load_last_n_days("synergy", 30)

In [11]:
lanes = {
    'TOP':{}, 
    'MIDDLE': {}, 
    'JUNGLE':{}, 
    'BOTTOM':{}, 
    'UTILITY': {}
}

for k, v in winrate_data.items():
    champion, lane = k.split('+')
    lanes.get(lane, {})[champion] = {
        'n': champion,
        'g': v['wins'] + v['losses'],
        'w': v['wins'] / (v['wins'] + v['losses']),
        's': {
            'TOP':{}, 
            'MIDDLE': {}, 
            'JUNGLE':{}, 
            'BOTTOM':{}, 
            'UTILITY': {}
        }
    }

for k, v in synergy_data.items():
    champion1, champion2, lane1, lane2 = k.split("+")
    if not lane1 or not lane2: continue
    if lane1 == lane2: continue
    if champion1 == champion2: continue
    
    if champion1 not in lanes[lane1]: continue
    if champion2 not in lanes[lane2]: continue

    
    average_winrate = (lanes[lane1][champion1]['w'] + lanes[lane2][champion2]['w']) / 2
    
    lanes[lane1][champion1]['s'][lane2][champion2] = {
        'g': v['wins'] + v['losses'],
        'w': v['wins'] / (v['wins'] + v['losses']) - average_winrate,
    }
    
    lanes[lane2][champion2]['s'][lane1][champion1] = {
        'g': v['wins'] + v['losses'],
        'w': v['wins'] / (v['wins'] + v['losses']) - average_winrate,
    }


In [ ]:
from collections import defaultdict
import itertools

# Preprocess
champion_role_pool = [(champ, role) for role, champions in lanes.items() for champ in champions]
games_played = { (champ, role): lanes[role][champ]['g'] for champ, role in champion_role_pool }

# Build all inter-role synergy pairs and reverse index
required_synergy_pairs = set()
pairs_by_champion = defaultdict(set)

for (c1, r1), (c2, r2) in itertools.combinations(champion_role_pool, 2):
    if c1 == c2 or r1 == r2:
        continue
    pair = ((c1, r1), (c2, r2))
    required_synergy_pairs.add(pair)
    pairs_by_champion[(c1, r1)].add(pair)
    pairs_by_champion[(c2, r2)].add(pair)

# Function to check if a synergy is satisfied
def is_synergy_ok(pair1, pair2):
    (c1, r1) = pair1
    (c2, r2) = pair2
    return (
        c2 in lanes[r1][c1]['s'][r2] and
        c1 in lanes[r2][c2]['s'][r1]
    )


# Initial unsatisfied pair tracking
unsatisfied_synergy_pairs = {
    pair for pair in required_synergy_pairs if not is_synergy_ok(*pair)
}

unsatisfied_count = defaultdict(int)
for (c1, r1), (c2, r2) in unsatisfied_synergy_pairs:
    unsatisfied_count[(c1, r1)] += 1
    unsatisfied_count[(c2, r2)] += 1

removed_champions = set()

while unsatisfied_synergy_pairs:
    # Find worst offender
    worst_offender = max(
        unsatisfied_count.items(),
        key=lambda item: (item[1], -games_played.get(item[0], 1e9))
    )[0]

    removed_champions.add(worst_offender)
    champ_to_remove, role_to_remove = worst_offender
    del lanes[role_to_remove][champ_to_remove]

    # Update affected pairs only
    affected_pairs = pairs_by_champion.pop(worst_offender, set())
    for pair in affected_pairs:
        if pair not in required_synergy_pairs:
            continue
        required_synergy_pairs.remove(pair)
        unsatisfied_synergy_pairs.discard(pair)

        for cr in pair:
            if cr != worst_offender:
                pairs_by_champion[cr].discard(pair)
                unsatisfied_count[cr] = max(0, unsatisfied_count[cr] - 1)

    del unsatisfied_count[worst_offender]

for lane, champions in lanes.items():
    for champion in champions.values():
        for slane, schampions in champion['s'].items():
            for schampion in list(schampions.keys()):
                if schampion not in lanes[slane]:
                    del champion['s'][slane][schampion]

print(f"Remaining champions: {sum(len(champs) for champs in lanes.values())}")


Remaining champions: 228


In [14]:
def verify_synergy_coverage(lanes):
    """
    Checks whether all remaining champions in `lanes` have full synergy data with
    every other champion of a different name and lane.

    Returns:
        success (bool): True if all required synergy pairs exist.
        missing (list): List of missing synergy tuples (champ1, lane1, champ2, lane2)
    """
    remaining = [(c, l) for l in lanes for c in lanes[l]]
    missing = []

    for i in range(len(remaining)):
        champ1, lane1 = remaining[i]
        for j in range(i + 1, len(remaining)):
            champ2, lane2 = remaining[j]
            if champ1 == champ2 or lane1 == lane2:
                continue

            # Check both directions
            valid = (
                champ2 in lanes[lane1][champ1]['s'][lane2] and
                champ1 in lanes[lane2][champ2]['s'][lane1]
            )
            if not valid:
                missing.append((champ1, lane1, champ2, lane2))

    if not missing:
        print("✅ Full synergy coverage verified.")
        return True, []
    else:
        print(f"❌ Missing {len(missing)} synergy entries.")
        for entry in missing[:10]:
            print("  Missing:", entry)
        if len(missing) > 10:
            print("  ...more not shown.")
        return False, missing

success, missing_synergies = verify_synergy_coverage(lanes)
assert success, "Some synergy pairs are missing!"


✅ Full synergy coverage verified.


In [15]:
# Requirements
import numpy as np
from scipy.stats import norm
import pandas as pd
import IPython.display as disp

# --- 1. Z-score bucket classifier
def make_zscore_bucket_classifier(values):
    mean = np.mean(values)
    std = np.std(values)
    z1 = norm.ppf(0.40)
    z2 = norm.ppf(0.70)
    z3 = norm.ppf(0.90)

    def classify(value):
        if std == 0:
            return 0
        z = (value - mean) / std
        if z < z1:
            return 0
        elif z < z2:
            return 1
        elif z < z3:
            return 2
        else:
            return 4  # note: was 4 in your original, using 0-3 range here

    return classify

# --- 2. Apply z-score bucketing to champion strength
champ_values = [champion['w'] for champions in lanes.values() for champion in champions.values()]
strength_score = make_zscore_bucket_classifier(champ_values)

for champions in lanes.values():
    for champion in champions.values():
        champion['p'] = strength_score(champion['w'])
        del champion['w']
        del champion['g']

# --- 3. Apply z-score bucketing to synergy scores
synergy_values = [
    schampion['w']
    for champions in lanes.values()
    for champion in champions.values()
    for schampions in champion['s'].values()
    for schampion in schampions.values()
]
synergy_score = make_zscore_bucket_classifier(synergy_values)

for champions in lanes.values():
    for champion in champions.values():
        for schampions in champion['s'].values():
            for schampion in schampions.values():
                schampion['p'] = synergy_score(schampion['w'])
                del schampion['w']
                del schampion['g']

# --- 4. Synergy matrix builder
def synergy_matrix(lanes, lane_a, lane_b, default="–"):
    rows = sorted(lanes[lane_a])
    cols = sorted(lanes[lane_b])
    data = []
    for champ_a in rows:
        row_vals = []
        for champ_b in cols:
            try:
                row_vals.append(lanes[lane_a][champ_a]["s"][lane_b][champ_b]["p"])
            except KeyError:
                row_vals.append(default)
        data.append(row_vals)
    return pd.DataFrame(data, index=rows, columns=cols)

# --- 5. Display all cross-lane synergy tables
for lane_a in lanes:
    for lane_b in lanes:
        if lane_a == lane_b:
            continue
        if lane_a > lane_b:
            continue
        matrix = synergy_matrix(lanes, lane_a, lane_b)
        disp.display(matrix.style.set_caption(f"{lane_a} × {lane_b} synergy buckets (0=weak … 3=strong)"))

with open('data.json', 'w') as f:
    json.dump(lanes, f, indent=2)

,Alistar,Bard,Blitzcrank,Brand,Braum,Elise,FiddleSticks,Janna,Karma,Leona,Lulu,Lux,Maokai,Mel,Milio,Morgana,Nami,Nautilus,Neeko,Pantheon,Poppy,Pyke,Rakan,Rell,Renata,Senna,Seraphine,Shaco,Shen,Sona,Soraka,Swain,TahmKench,Taric,Thresh,Velkoz,Xerath,Yuumi,Zilean,Zoe,Zyra
Aatrox,0,1,2,1,1,2,1,1,1,0,1,2,2,2,2,2,1,1,1,1,0,1,1,1,1,2,2,1,4,1,2,0,0,0,1,1,2,0,2,4,1
Akali,2,1,1,4,2,0,4,0,0,0,1,1,0,0,2,1,0,2,0,0,4,0,0,0,0,0,2,0,0,1,1,2,4,0,0,0,2,1,1,0,2
Ambessa,1,1,1,2,0,1,1,1,2,0,0,0,1,1,2,2,1,2,0,2,0,0,1,1,1,0,2,0,0,1,1,1,1,1,2,2,1,0,2,2,1
Aurora,0,0,0,0,0,1,2,1,4,0,2,0,0,1,2,0,0,2,2,1,2,0,4,0,4,2,0,1,4,0,4,0,2,0,0,1,4,1,4,4,0
Camille,2,2,2,1,0,0,0,1,1,0,0,2,1,0,0,1,1,0,0,1,2,1,1,1,1,1,1,4,2,1,1,0,0,0,0,2,0,1,4,4,2
Chogath,1,1,2,0,1,0,1,2,1,2,0,1,2,0,2,2,1,1,1,2,0,1,2,4,1,1,1,4,0,0,0,2,1,1,1,0,1,2,0,2,2
Darius,0,1,1,0,1,1,1,1,1,1,1,2,1,1,0,1,1,1,0,1,0,1,1,1,1,1,0,1,2,0,1,1,1,2,1,0,1,1,1,4,1
DrMundo,1,0,1,1,1,1,0,1,1,1,1,2,1,2,0,1,0,1,4,0,0,1,2,0,4,1,1,2,1,1,0,1,2,0,0,0,0,0,2,0,0
Fiora,0,2,1,1,1,1,1,1,2,1,1,0,4,1,1,0,0,1,2,1,0,1,1,0,2,1,0,2,2,0,0,1,0,0,1,1,1,0,1,1,0
Gangplank,1,0,0,2,1,1,0,2,1,1,0,0,2,2,1,0,1,2,0,2,1,1,0,2,0,2,2,1,0,0,0,1,4,0,0,1,2,1,0,0,0


,Aatrox,Akali,Ambessa,Aurora,Camille,Chogath,Darius,DrMundo,Fiora,Gangplank,Garen,Gnar,Gragas,Gwen,Heimerdinger,Illaoi,Irelia,Jax,Jayce,KSante,Kayle,Kennen,Kled,Malphite,MonkeyKing,Mordekaiser,Nasus,Olaf,Ornn,Pantheon,Poppy,Quinn,Renekton,Riven,Rumble,Ryze,Sett,Shen,Singed,Sion,Sylas,TahmKench,Teemo,Trundle,Tryndamere,Urgot,Varus,Vayne,Vladimir,Volibear,Warwick,Yasuo,Yone,Yorick
Ahri,1,1,1,1,0,2,1,0,1,4,1,1,1,0,0,1,1,0,1,2,1,1,1,0,0,1,1,0,1,1,1,0,0,1,4,1,1,0,0,1,4,1,1,1,1,0,2,4,0,2,1,2,1,1
Akali,1,–,1,0,2,1,1,2,1,1,0,1,1,2,0,1,0,1,1,2,0,0,0,1,1,0,1,0,1,0,0,0,1,1,1,0,1,0,1,0,1,0,0,0,1,0,4,0,2,1,0,0,2,1
Akshan,1,1,2,4,0,2,0,1,0,2,1,2,1,0,2,1,0,2,0,1,1,1,4,0,4,2,1,2,2,0,1,0,1,1,1,2,1,0,0,1,0,0,1,2,0,0,2,1,1,2,1,0,1,1
Anivia,2,1,1,0,2,0,0,1,0,0,0,2,0,0,0,1,0,1,2,1,0,1,1,1,0,1,2,0,1,4,0,0,1,0,1,0,1,0,1,1,1,4,0,4,0,1,4,0,1,2,4,2,1,1
Annie,1,2,1,1,0,1,1,1,1,1,1,1,0,0,2,0,2,1,2,2,2,0,0,0,0,1,0,1,4,0,1,0,1,0,2,0,0,1,0,0,0,0,1,2,1,1,4,2,1,4,0,4,0,0
AurelionSol,0,2,1,4,2,0,2,1,1,0,1,4,0,1,1,0,4,0,0,4,1,2,4,4,1,0,2,0,0,1,1,4,1,1,4,1,0,1,2,1,2,4,1,1,0,0,0,1,1,1,0,1,1,0
Aurora,1,1,0,–,2,2,0,0,2,4,1,1,0,0,1,1,1,2,1,2,1,0,2,2,0,2,1,0,0,0,1,0,4,0,0,4,1,2,4,1,1,0,1,1,2,1,0,0,2,1,1,1,2,1
Azir,2,0,1,4,1,1,1,2,1,1,1,0,1,2,2,1,2,2,2,1,1,2,4,0,2,0,1,1,1,1,2,0,1,2,4,1,1,0,0,0,2,0,0,0,0,1,0,1,0,1,1,1,1,1
Cassiopeia,0,4,2,2,0,0,0,4,2,0,2,0,0,0,4,2,1,0,2,2,4,2,4,1,2,2,1,1,0,0,2,1,1,1,1,0,1,0,0,0,4,4,1,0,2,0,4,4,0,0,4,2,0,0
Chogath,0,0,1,2,1,–,1,0,0,1,0,0,0,2,0,0,0,0,2,4,1,1,1,0,1,0,1,1,0,0,2,0,1,0,0,0,2,0,4,0,0,0,2,1,2,1,1,0,0,1,0,2,0,1


,Alistar,Bard,Blitzcrank,Brand,Braum,Elise,FiddleSticks,Janna,Karma,Leona,Lulu,Lux,Maokai,Mel,Milio,Morgana,Nami,Nautilus,Neeko,Pantheon,Poppy,Pyke,Rakan,Rell,Renata,Senna,Seraphine,Shaco,Shen,Sona,Soraka,Swain,TahmKench,Taric,Thresh,Velkoz,Xerath,Yuumi,Zilean,Zoe,Zyra
Ahri,1,1,1,0,0,0,1,1,1,1,0,2,0,0,1,2,1,0,1,1,2,1,1,1,1,1,1,1,0,1,1,1,1,0,1,1,2,0,2,2,1
Akali,0,1,0,1,1,0,0,2,0,0,0,2,1,2,1,2,1,0,0,0,1,1,1,0,2,1,1,2,1,1,0,1,0,0,1,1,0,1,0,4,1
Akshan,0,0,1,0,1,0,1,0,1,0,1,1,2,2,0,0,0,2,4,2,0,0,1,0,0,1,0,1,0,0,2,0,1,2,1,1,0,2,0,0,2
Anivia,0,1,0,1,1,1,0,1,1,0,0,0,0,2,0,1,0,1,4,1,0,2,1,1,0,1,4,1,4,1,1,1,4,4,1,1,2,0,2,0,0
Annie,1,1,2,2,0,1,1,0,1,1,1,1,0,0,2,2,0,2,0,0,0,0,1,1,2,1,0,0,4,0,1,0,0,2,0,0,1,0,2,0,0
AurelionSol,0,2,1,0,0,0,0,2,2,1,0,2,1,0,1,2,2,0,0,2,0,0,0,0,1,1,0,4,1,2,1,4,1,2,1,2,1,2,0,4,1
Aurora,2,2,1,0,0,0,1,2,1,2,2,1,0,4,4,0,1,1,0,2,0,1,1,2,0,1,1,4,0,0,1,0,0,2,1,2,4,1,0,4,0
Azir,1,0,1,0,2,2,2,2,0,1,0,1,0,1,1,0,1,2,1,0,1,1,1,1,0,2,0,4,1,1,1,2,0,4,1,0,2,2,1,4,4
Cassiopeia,0,1,4,4,0,2,2,0,1,0,4,4,1,0,0,0,0,0,4,0,4,0,1,0,0,1,2,4,4,1,2,0,2,1,1,0,1,0,1,0,4
Chogath,1,2,1,0,1,1,0,1,2,4,0,1,0,0,0,0,0,0,2,2,0,0,0,1,2,1,2,4,2,2,4,0,0,0,1,1,0,2,0,4,1


,Aatrox,Akali,Ambessa,Aurora,Camille,Chogath,Darius,DrMundo,Fiora,Gangplank,Garen,Gnar,Gragas,Gwen,Heimerdinger,Illaoi,Irelia,Jax,Jayce,KSante,Kayle,Kennen,Kled,Malphite,MonkeyKing,Mordekaiser,Nasus,Olaf,Ornn,Pantheon,Poppy,Quinn,Renekton,Riven,Rumble,Ryze,Sett,Shen,Singed,Sion,Sylas,TahmKench,Teemo,Trundle,Tryndamere,Urgot,Varus,Vayne,Vladimir,Volibear,Warwick,Yasuo,Yone,Yorick
Amumu,1,0,2,1,1,2,1,4,2,0,2,2,1,2,0,1,0,2,1,1,0,0,4,1,1,0,4,0,2,0,0,1,2,1,2,0,1,0,2,1,1,1,1,0,1,1,1,0,1,1,1,2,1,0
Belveth,1,0,1,1,1,1,1,1,1,1,1,0,0,0,2,2,1,1,1,2,0,1,0,2,1,1,0,0,1,2,1,0,1,0,2,4,1,1,1,0,0,0,1,0,0,0,2,0,2,2,0,2,1,0
Brand,1,0,4,4,4,4,1,2,1,2,2,1,1,1,4,4,4,0,2,0,0,2,4,1,4,4,0,4,2,1,4,2,1,1,4,4,0,0,4,0,2,4,0,1,2,0,0,4,0,0,0,0,0,1
Briar,1,0,1,0,1,0,1,0,0,4,0,1,1,2,1,1,0,1,2,1,1,1,0,0,4,1,0,0,0,0,0,0,1,2,4,0,1,0,4,1,1,0,1,1,2,1,0,2,1,1,2,1,2,1
Darius,1,0,1,4,2,0,–,1,1,1,2,2,0,1,0,1,4,1,2,1,2,0,2,0,0,0,1,2,0,0,1,0,2,0,2,1,1,2,0,2,0,0,2,4,0,1,1,0,0,0,4,1,1,0
Diana,2,0,1,2,0,1,1,0,0,1,1,1,1,1,0,1,1,0,1,2,2,1,1,0,1,2,0,0,1,1,0,0,2,1,1,0,0,1,2,1,1,0,1,2,1,0,0,2,2,2,1,1,0,0
DrMundo,1,1,0,0,0,1,2,–,0,1,0,0,0,1,1,2,1,1,2,0,1,4,2,2,1,0,0,0,4,1,4,1,1,1,1,1,1,2,0,1,4,4,1,4,0,0,0,4,0,1,0,2,1,2
Ekko,2,0,1,4,2,0,1,1,0,2,1,1,2,1,0,1,1,0,2,0,2,0,1,1,1,1,0,2,1,0,1,2,0,0,1,1,1,0,1,0,0,2,0,1,1,0,4,0,1,1,1,1,1,1
Elise,1,2,1,0,0,2,1,2,0,1,0,0,1,4,2,4,0,0,1,0,1,2,0,1,1,1,2,0,0,0,1,0,1,0,1,1,0,0,1,0,0,0,4,1,1,0,1,0,1,0,1,0,2,0
Evelynn,1,0,1,1,2,4,0,1,0,2,0,0,1,0,0,0,0,0,0,0,1,4,4,1,0,2,0,2,0,0,0,0,2,0,4,4,1,0,2,0,0,0,0,1,1,0,4,0,0,0,2,4,1,1


,Ahri,Akali,Akshan,Anivia,Annie,AurelionSol,Aurora,Azir,Cassiopeia,Chogath,Corki,Diana,Ekko,Fizz,Galio,Hwei,Irelia,Jayce,Kassadin,Katarina,Kayle,Kennen,Leblanc,Lissandra,Lux,Malphite,Malzahar,Mel,Naafiri,Orianna,Pantheon,Qiyana,Ryze,Smolder,Swain,Sylas,Syndra,Taliyah,Talon,Tristana,TwistedFate,Veigar,Velkoz,Vex,Viktor,Vladimir,Xerath,Yasuo,Yone,Zed,Zoe
Amumu,2,1,0,1,1,2,2,1,4,1,0,0,1,0,2,1,0,4,0,0,1,1,1,4,2,2,0,0,1,1,0,1,0,4,2,0,0,1,1,0,2,1,2,0,2,2,1,1,2,1,4
Belveth,2,0,1,1,0,0,0,0,4,2,4,1,1,0,1,1,0,4,1,1,1,1,0,0,2,0,1,0,2,2,2,4,1,4,0,0,0,2,1,2,0,1,0,0,0,0,1,1,1,2,0
Brand,2,2,4,0,0,0,0,4,4,0,0,4,0,0,1,0,4,0,0,4,0,1,0,0,1,4,2,0,4,2,1,1,0,1,2,2,1,0,1,4,4,2,4,4,0,2,0,1,4,1,2
Briar,1,0,0,4,1,1,0,1,1,0,1,0,0,0,1,2,2,2,2,2,0,4,0,0,2,0,2,0,4,2,0,0,1,4,0,1,1,0,0,0,1,1,0,0,1,1,4,0,1,0,2
Darius,0,0,2,1,1,1,4,1,1,0,0,1,0,0,0,1,0,4,1,1,0,2,0,4,2,4,0,1,0,2,4,0,0,0,0,2,0,0,0,4,2,1,0,1,2,1,0,1,4,2,1
Diana,0,1,0,0,2,0,2,2,1,0,1,–,0,4,1,2,0,1,0,1,0,2,0,0,4,1,0,1,0,0,2,2,0,2,0,2,1,0,0,0,1,4,0,1,1,2,1,1,1,2,2
DrMundo,0,2,4,4,0,1,2,0,0,0,0,1,4,0,1,0,1,0,2,1,1,4,1,0,1,4,0,4,0,1,0,4,2,0,0,0,1,0,0,0,1,4,2,1,0,1,1,1,1,2,0
Ekko,1,0,1,0,0,0,0,2,4,1,1,0,–,0,0,2,1,0,4,1,0,0,1,4,2,1,0,2,0,2,0,1,0,0,2,1,1,2,1,0,1,1,4,0,2,0,1,1,0,1,1
Elise,0,0,1,1,1,1,1,4,0,0,0,0,1,1,0,0,0,0,1,0,2,0,0,4,1,0,0,0,0,4,4,2,0,4,0,0,0,1,0,1,0,2,0,1,0,1,4,1,1,0,1
Evelynn,0,1,1,2,0,4,1,1,4,0,0,0,1,0,1,0,0,2,0,0,0,0,0,1,0,0,0,1,2,0,1,1,1,0,0,0,1,4,2,1,0,1,0,2,2,0,0,1,1,0,0


,Alistar,Bard,Blitzcrank,Brand,Braum,Elise,FiddleSticks,Janna,Karma,Leona,Lulu,Lux,Maokai,Mel,Milio,Morgana,Nami,Nautilus,Neeko,Pantheon,Poppy,Pyke,Rakan,Rell,Renata,Senna,Seraphine,Shaco,Shen,Sona,Soraka,Swain,TahmKench,Taric,Thresh,Velkoz,Xerath,Yuumi,Zilean,Zoe,Zyra
Amumu,1,4,2,0,1,2,0,2,1,1,0,2,1,1,1,2,1,1,2,2,0,1,2,1,1,0,1,1,0,1,1,1,0,0,1,1,2,1,1,1,2
Belveth,0,0,1,2,1,0,2,1,1,0,1,4,4,2,1,0,0,1,2,2,0,0,2,0,1,1,2,4,0,0,0,1,1,0,1,1,4,1,4,1,1
Brand,0,1,1,–,1,0,2,0,0,4,2,1,0,1,0,0,4,1,4,2,0,0,1,0,1,0,0,0,0,1,1,4,0,0,2,1,0,2,1,4,4
Briar,1,1,2,0,1,1,0,1,0,1,0,1,2,0,0,1,1,0,0,2,1,1,2,1,0,0,1,1,2,1,0,0,4,2,1,1,0,1,1,0,1
Darius,0,0,0,0,2,1,1,0,1,0,0,0,1,2,1,2,1,2,2,0,2,2,1,1,4,1,0,4,4,2,2,4,0,0,1,1,0,0,0,0,0
Diana,1,1,1,4,0,0,1,0,1,1,1,2,1,0,1,1,1,1,0,1,2,2,0,1,0,0,1,1,4,1,1,0,0,1,1,1,1,1,0,2,0
DrMundo,0,0,0,1,0,1,1,2,1,0,1,4,1,0,2,0,0,0,1,1,0,1,2,0,2,1,2,2,0,1,0,2,0,2,0,1,1,1,0,0,0
Ekko,1,2,0,0,0,1,0,1,1,1,1,2,2,2,1,2,1,1,0,2,1,1,1,1,1,1,0,0,0,0,0,1,2,1,1,0,0,0,2,4,0
Elise,0,0,0,0,0,–,0,1,1,1,0,1,4,1,1,1,0,1,1,0,0,0,4,0,0,1,1,4,0,2,1,1,0,0,1,4,0,1,1,1,0
Evelynn,1,1,0,4,0,0,0,1,2,0,0,2,0,0,0,1,1,1,2,2,0,2,0,0,0,2,1,4,0,0,0,1,2,0,1,0,2,0,1,1,1


,Aatrox,Akali,Ambessa,Aurora,Camille,Chogath,Darius,DrMundo,Fiora,Gangplank,Garen,Gnar,Gragas,Gwen,Heimerdinger,Illaoi,Irelia,Jax,Jayce,KSante,Kayle,Kennen,Kled,Malphite,MonkeyKing,Mordekaiser,Nasus,Olaf,Ornn,Pantheon,Poppy,Quinn,Renekton,Riven,Rumble,Ryze,Sett,Shen,Singed,Sion,Sylas,TahmKench,Teemo,Trundle,Tryndamere,Urgot,Varus,Vayne,Vladimir,Volibear,Warwick,Yasuo,Yone,Yorick
Aphelios,1,0,1,1,1,0,1,0,1,1,2,1,0,0,0,1,0,1,2,0,1,0,0,0,1,1,1,0,1,0,2,1,0,0,0,0,1,0,1,1,2,0,1,0,0,0,1,1,0,1,0,1,1,1
Ashe,1,0,1,1,0,0,1,1,1,1,1,0,0,1,1,4,1,1,1,2,1,2,1,0,1,2,1,0,1,1,0,1,4,0,2,1,0,0,0,1,0,1,0,0,1,1,2,2,1,1,1,0,2,1
Brand,0,0,0,4,2,0,0,0,0,1,1,0,4,4,0,0,2,2,2,4,2,1,2,2,0,1,2,0,0,0,0,0,1,0,0,0,2,1,4,0,0,4,2,1,0,1,1,0,0,4,1,0,0,1
Caitlyn,1,0,1,0,1,2,1,1,1,1,0,2,1,1,1,1,1,1,1,1,1,1,0,1,1,1,2,0,1,2,1,0,1,1,1,1,1,0,2,0,2,0,1,1,1,0,0,2,0,1,1,1,2,1
Corki,4,0,2,1,0,1,1,0,1,0,0,0,1,0,1,1,0,1,1,1,1,0,1,0,0,1,2,2,0,0,0,0,2,1,0,0,0,1,1,0,0,2,2,1,1,1,1,0,2,4,4,2,2,1
Draven,1,1,0,0,0,2,1,1,0,2,1,1,0,1,0,2,1,1,0,1,0,0,0,0,0,1,0,0,1,1,0,0,1,1,1,2,0,1,1,0,0,1,1,4,1,1,2,1,0,2,1,2,1,0
Ezreal,1,1,1,0,1,2,1,1,0,1,2,1,0,1,0,0,0,1,1,1,1,2,0,1,4,1,1,1,1,1,1,2,1,1,2,1,1,1,0,1,0,0,2,1,1,1,4,2,2,1,0,2,1,0
Hwei,0,2,1,0,0,0,0,1,2,1,1,0,1,0,0,0,1,0,1,1,2,0,0,0,1,1,1,1,1,4,0,1,4,0,1,4,2,4,0,0,4,0,0,1,1,1,1,4,0,4,0,2,0,0
Jhin,2,2,1,0,1,1,0,1,1,1,1,0,1,1,1,1,1,0,1,1,1,1,1,1,1,2,1,1,1,1,1,0,1,1,2,2,1,1,1,1,0,1,1,2,0,0,4,0,0,1,0,1,1,1
Jinx,1,1,1,0,1,1,0,1,1,1,0,1,1,0,0,2,1,1,1,2,0,1,1,1,1,1,2,0,1,0,1,0,1,1,1,0,1,0,1,1,0,0,1,1,0,0,1,1,1,1,0,4,1,0


,Ahri,Akali,Akshan,Anivia,Annie,AurelionSol,Aurora,Azir,Cassiopeia,Chogath,Corki,Diana,Ekko,Fizz,Galio,Hwei,Irelia,Jayce,Kassadin,Katarina,Kayle,Kennen,Leblanc,Lissandra,Lux,Malphite,Malzahar,Mel,Naafiri,Orianna,Pantheon,Qiyana,Ryze,Smolder,Swain,Sylas,Syndra,Taliyah,Talon,Tristana,TwistedFate,Veigar,Velkoz,Vex,Viktor,Vladimir,Xerath,Yasuo,Yone,Zed,Zoe
Aphelios,0,0,0,1,1,2,0,0,0,0,1,0,1,0,0,1,0,1,1,0,1,1,0,1,1,0,0,1,0,0,2,0,0,4,1,1,1,2,2,1,1,1,0,0,0,0,1,1,1,0,2
Ashe,1,1,0,2,1,2,2,1,2,0,1,1,1,1,0,1,2,2,1,1,1,2,1,1,1,4,1,0,0,1,1,1,0,1,0,1,1,0,1,2,2,1,1,1,1,1,2,1,1,0,0
Brand,1,2,0,4,0,4,4,1,4,0,0,0,2,1,2,0,4,0,4,2,1,4,4,0,2,4,1,0,0,0,4,0,0,2,0,4,1,0,0,0,4,2,4,0,4,2,0,0,1,1,4
Caitlyn,1,1,0,0,1,0,1,1,2,2,0,1,0,1,1,1,1,2,1,1,1,0,1,1,2,2,1,1,1,1,1,1,1,2,0,1,1,1,0,1,1,1,1,0,1,0,0,1,1,1,1
Corki,1,1,0,1,1,1,0,1,2,1,–,1,0,0,1,0,2,0,4,0,0,2,1,0,2,2,1,2,1,0,4,1,1,0,4,2,0,1,0,1,2,4,0,0,0,0,4,2,1,1,2
Draven,0,0,0,0,1,2,2,0,2,0,0,1,0,1,1,1,1,1,0,1,2,2,0,0,1,2,1,1,0,0,0,0,1,0,0,1,1,2,1,1,1,1,0,1,1,0,1,1,1,0,0
Ezreal,1,0,1,0,1,0,2,1,1,2,2,0,1,1,1,1,0,2,1,0,1,1,1,2,1,1,2,2,0,1,2,1,1,0,0,1,1,0,1,1,1,2,1,2,1,1,1,1,1,1,0
Hwei,0,0,2,0,0,0,2,0,0,4,1,0,0,0,0,–,0,0,4,1,1,0,0,0,1,2,0,0,2,2,1,2,0,2,1,2,2,4,1,1,2,2,0,2,0,0,0,1,0,0,0
Jhin,1,1,1,1,1,2,1,2,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,2,1,4,0,1,1,1,0,1,1,1,1,1,1,0,0,1,1,1,1
Jinx,1,1,1,1,1,1,1,1,0,0,2,1,0,1,0,1,1,1,0,0,1,0,1,0,1,0,0,1,0,1,1,1,1,2,0,1,1,1,0,1,1,1,1,0,1,1,1,0,1,0,1


,Amumu,Belveth,Brand,Briar,Darius,Diana,DrMundo,Ekko,Elise,Evelynn,FiddleSticks,Gragas,Graves,Gwen,Hecarim,Ivern,JarvanIV,Karthus,Kayn,Khazix,Kindred,LeeSin,Lillia,MasterYi,MonkeyKing,Naafiri,Nidalee,Nocturne,Nunu,Pantheon,Poppy,Rammus,RekSai,Rengar,Sejuani,Shaco,Shyvana,Skarner,Taliyah,Talon,Udyr,Vi,Viego,Volibear,Warwick,XinZhao,Yorick,Zac,Zed,Zyra
Aphelios,1,1,0,0,1,1,1,1,0,0,0,0,0,0,0,1,1,1,1,0,0,1,0,0,1,0,1,0,0,1,4,1,0,0,0,1,0,1,0,1,1,1,1,1,1,1,1,1,0,1
Ashe,1,1,2,2,0,0,0,1,0,0,4,0,0,1,1,1,1,0,1,0,1,1,1,0,0,1,1,1,0,1,1,1,0,1,0,1,1,0,0,1,0,1,1,0,2,1,0,1,2,1
Brand,2,4,–,1,4,0,1,4,1,4,0,0,1,0,1,0,0,0,1,0,0,4,0,0,4,1,1,2,1,4,4,0,0,1,1,0,4,4,0,0,0,1,1,0,0,2,4,0,0,1
Caitlyn,2,1,0,1,2,1,2,1,0,1,1,0,1,2,0,0,1,2,1,1,0,1,1,0,0,1,2,1,1,1,0,2,1,1,0,1,1,1,1,1,1,1,1,0,2,1,1,2,1,1
Corki,1,1,0,2,1,1,2,4,0,1,1,2,1,1,1,0,0,0,1,0,0,0,2,0,1,2,2,1,1,1,0,4,1,2,0,1,0,1,0,0,0,1,1,1,1,1,2,1,1,1
Draven,1,0,1,0,0,0,1,0,0,1,0,2,1,2,1,0,1,0,1,1,1,1,1,1,1,1,2,0,0,1,1,1,1,1,1,1,2,1,4,0,0,2,1,1,1,1,1,0,0,2
Ezreal,2,1,2,0,0,1,1,1,1,1,0,2,1,2,1,1,0,0,1,1,1,1,1,1,0,1,2,1,1,1,0,0,2,2,1,1,2,1,1,1,1,1,1,1,1,1,1,1,0,1
Hwei,2,0,1,1,4,2,1,0,4,0,4,0,1,1,0,1,2,0,0,2,0,0,2,0,0,0,1,0,0,2,2,4,4,0,0,1,0,2,0,2,2,1,1,2,0,0,4,1,1,0
Jhin,1,1,1,1,0,1,2,2,1,1,0,1,1,2,1,0,1,2,1,1,0,1,1,0,1,1,2,1,1,1,1,1,1,1,2,1,1,1,0,0,0,1,1,1,0,1,1,1,0,1
Jinx,1,1,4,1,1,1,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,2,0,1,1,1,2,2,1,1,1,0,1,1,1,0,0,1,1,0,1,1,1,0,1,1,1,0,1,2


,Alistar,Bard,Blitzcrank,Brand,Braum,Elise,FiddleSticks,Janna,Karma,Leona,Lulu,Lux,Maokai,Mel,Milio,Morgana,Nami,Nautilus,Neeko,Pantheon,Poppy,Pyke,Rakan,Rell,Renata,Senna,Seraphine,Shaco,Shen,Sona,Soraka,Swain,TahmKench,Taric,Thresh,Velkoz,Xerath,Yuumi,Zilean,Zoe,Zyra
Aphelios,0,2,0,2,0,1,0,0,1,1,1,2,0,4,0,1,0,2,0,0,1,0,0,0,0,1,2,0,4,0,1,1,0,0,1,1,2,1,0,0,1
Ashe,2,1,2,2,0,1,1,1,1,1,0,1,1,1,0,1,0,1,1,1,1,1,1,1,1,1,1,0,0,1,1,2,1,2,1,0,0,1,1,1,1
Brand,0,2,0,–,1,0,4,2,1,4,4,0,0,2,0,4,1,0,1,2,0,1,1,0,0,1,4,2,4,0,0,0,4,4,2,1,2,2,0,0,1
Caitlyn,0,1,1,0,1,1,0,1,1,2,1,1,2,0,1,1,1,0,2,1,0,1,1,2,1,1,1,0,0,1,0,2,1,1,1,0,1,1,1,2,0
Corki,1,0,1,1,1,0,2,1,2,2,1,4,2,2,0,0,1,1,0,0,1,1,2,0,1,0,0,2,4,0,1,0,2,1,1,0,2,0,0,1,1
Draven,1,2,2,2,0,1,1,0,0,1,0,2,2,1,1,1,0,1,1,1,1,1,1,1,2,1,0,2,2,0,2,0,1,2,0,1,4,1,0,2,0
Ezreal,1,1,1,2,1,1,1,1,1,0,1,1,1,1,1,2,1,1,2,1,0,0,1,1,1,1,1,2,0,1,1,2,1,0,1,1,2,1,1,1,1
Hwei,1,1,2,4,2,0,0,2,1,0,0,0,1,0,0,1,1,2,4,1,0,1,1,0,2,0,1,0,0,2,1,0,2,4,0,1,1,0,1,4,0
Jhin,1,1,1,1,1,1,0,1,1,1,1,1,2,2,1,1,1,1,0,1,1,1,1,1,2,1,1,1,1,1,1,0,1,0,1,1,1,1,1,2,1
Jinx,1,1,2,0,1,0,1,0,1,1,1,2,1,2,1,0,0,0,2,1,0,1,1,1,1,1,1,1,0,0,1,2,1,1,1,1,2,0,2,4,1


In [16]:
from collections import Counter
import pandas as pd

def print_bucket_stats(name, values):
    counter = Counter(values)
    total = sum(counter.values())
    data = {
        "Bucket": list(range(5)),
        "Count": [counter.get(i, 0) for i in range(5)],
        "Percentage": [round(counter.get(i, 0) / total * 100, 2) for i in range(5)]
    }
    df = pd.DataFrame(data)
    df.set_index("Bucket", inplace=True)
    disp.display(df.style.set_caption(f"{name} Bucket Distribution"))

# --- Gather values
champ_buckets = [champion['p'] for champions in lanes.values() for champion in champions.values()]
synergy_buckets = [
    schampion['p']
    for champions in lanes.values()
    for champion in champions.values()
    for schampions in champion['s'].values()
    for schampion in schampions.values()
]

# --- Print stats
print_bucket_stats("Champion Strength", champ_buckets)
print_bucket_stats("Synergy Score", synergy_buckets)


,Count,Percentage
Bucket,,
0,88,38.600000
1,79,34.650000
2,44,19.300000
3,0,0.000000
4,17,7.460000


,Count,Percentage
Bucket,,
0,14888,36.200000
1,16418,39.920000
2,6714,16.320000
3,0,0.000000
4,3108,7.560000


In [ ]:
# --- compute & sort champion “p‑plus‑synergy” scores
scores = []

for lane, champions in lanes.items():
    for champ in champions.values():
        score = 0
        for lane_synergies in champ['s'].values():
            synergy_ps = [t['p'] for t in lane_synergies.values()]
            # Avoid divide‑by‑zero if the list is empty
            avg_synergy = sum(synergy_ps) / len(synergy_ps) if synergy_ps else 0
            score += avg_synergy
        score += champ['p']
        scores.append((champ['n'], score))

# sort descending by score
scores.sort(key=lambda x: x[1], reverse=True)

# pretty‑print
for name, score in scores:
    print(f"{name:20} {score:8.3f}")

'https://ddragon.leagueoflegends.com/cdn/15.9.1/img/champion/Poppy.png'

Brand BOTTOM            9.471
Karthus BOTTOM          9.001
RekSai JUNGLE           8.308
Yasuo BOTTOM            8.307
Pantheon MIDDLE         8.246
KogMaw BOTTOM           8.069
Sylas TOP               7.974
Seraphine BOTTOM        7.925
Anivia MIDDLE           7.915
Swain BOTTOM            7.529
Zoe UTILITY             7.524
Elise JUNGLE            7.473
FiddleSticks UTILITY    7.365
Lux BOTTOM              7.272
Nilah BOTTOM            7.243
Cassiopeia MIDDLE       7.178
Vayne TOP               7.108
Nidalee JUNGLE          7.066
Olaf TOP                7.058
Shaco UTILITY           7.039
Taliyah JUNGLE          7.026
Kindred JUNGLE          6.721
Varus TOP               6.693
Urgot TOP               6.535
Poppy TOP               6.482
Rumble TOP              6.410
MonkeyKing TOP          6.389
Heimerdinger TOP        6.385
Smolder MIDDLE          6.210
Corki MIDDLE            6.167
Belveth JUNGLE          6.156
Kled TOP                6.052
Velkoz MIDDLE           6.019
Yorick JUN

'https://ddragon.leagueoflegends.com/cdn/15.9.1/img/champion/Poppy.png'

In [19]:
duplicate = {}

for lane, champions in lanes.items():
    for champion in champions.keys():
        if champion in duplicate:
            duplicate[champion].append(lane)
        else:
            duplicate[champion] = [lane]

for k, v in duplicate.items():
    if len(v) > 1:
        print(k, v)

Darius ['TOP', 'JUNGLE']
Pantheon ['TOP', 'MIDDLE', 'JUNGLE', 'UTILITY']
Gwen ['TOP', 'JUNGLE']
Irelia ['TOP', 'MIDDLE']
Shen ['TOP', 'UTILITY']
Vayne ['TOP', 'BOTTOM']
MonkeyKing ['TOP', 'JUNGLE']
Akali ['TOP', 'MIDDLE']
DrMundo ['TOP', 'JUNGLE']
Volibear ['TOP', 'JUNGLE']
Poppy ['TOP', 'JUNGLE', 'UTILITY']
Vladimir ['TOP', 'MIDDLE']
Yone ['TOP', 'MIDDLE']
Yasuo ['TOP', 'MIDDLE', 'BOTTOM']
Kennen ['TOP', 'MIDDLE']
Kayle ['TOP', 'MIDDLE']
Yorick ['TOP', 'JUNGLE']
TahmKench ['TOP', 'UTILITY']
Chogath ['TOP', 'MIDDLE']
Ryze ['TOP', 'MIDDLE']
Malphite ['TOP', 'MIDDLE']
Jayce ['TOP', 'MIDDLE']
Warwick ['TOP', 'JUNGLE']
Varus ['TOP', 'BOTTOM']
Aurora ['TOP', 'MIDDLE']
Gragas ['TOP', 'JUNGLE']
Sylas ['TOP', 'MIDDLE']
Hwei ['MIDDLE', 'BOTTOM']
Zed ['MIDDLE', 'JUNGLE']
Ekko ['MIDDLE', 'JUNGLE']
Diana ['MIDDLE', 'JUNGLE']
Tristana ['MIDDLE', 'BOTTOM']
Taliyah ['MIDDLE', 'JUNGLE']
Xerath ['MIDDLE', 'UTILITY']
Mel ['MIDDLE', 'BOTTOM', 'UTILITY']
Naafiri ['MIDDLE', 'JUNGLE']
Swain ['MIDDLE', 'BOTT

In [ ]:
print(lanes)

{'TOP': {'Garen': {'n': 'Garen', 's': {'TOP': {}, 'MIDDLE': {'Viktor': {'p': 1}, 'Zed': {'p': 1}, 'Fizz': {'p': 2}, 'Malphite': {'p': 0}, 'Kayle': {'p': 0}, 'AurelionSol': {'p': 1}, 'Veigar': {'p': 2}, 'Talon': {'p': 0}, 'Katarina': {'p': 2}, 'Irelia': {'p': 2}, 'Diana': {'p': 0}, 'Yasuo': {'p': 1}, 'Leblanc': {'p': 1}, 'Ryze': {'p': 1}, 'Swain': {'p': 0}, 'TwistedFate': {'p': 1}, 'Azir': {'p': 1}, 'Qiyana': {'p': 1}, 'Hwei': {'p': 1}, 'Sylas': {'p': 1}, 'Chogath': {'p': 0}, 'Akali': {'p': 0}, 'Syndra': {'p': 1}, 'Galio': {'p': 1}, 'Mel': {'p': 1}, 'Yone': {'p': 2}, 'Ahri': {'p': 1}, 'Anivia': {'p': 0}, 'Xerath': {'p': 1}, 'Cassiopeia': {'p': 2}, 'Orianna': {'p': 1}, 'Lissandra': {'p': 4}, 'Zoe': {'p': 0}, 'Aurora': {'p': 1}, 'Vladimir': {'p': 0}, 'Annie': {'p': 1}, 'Akshan': {'p': 1}, 'Vex': {'p': 0}, 'Kennen': {'p': 0}, 'Lux': {'p': 1}, 'Ekko': {'p': 1}, 'Taliyah': {'p': 0}, 'Malzahar': {'p': 1}, 'Tristana': {'p': 0}, 'Naafiri': {'p': 1}, 'Corki': {'p': 1}, 'Jayce': {'p': 1}, 'Kassad